In [1]:
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from binance.client import Client
from binance import AsyncClient
import glob
import os
from pathlib import Path
import asyncio

In [2]:
# Load Environment Variables
load_dotenv()

# Gets MongoDB Connection String
MDB_CONNECTION_STRING = os.getenv('MDB_CONNECTION_STRING')

# Gets Binance Api Key and Api Secret
BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')
BINANCE_API_SECRET = os.getenv('BINANCE_API_SECRET')

In [3]:
# Function to connect to the Mongo DB
def get_database():
    try:
        client = MongoClient(MDB_CONNECTION_STRING)
        db = client["project-02"]
        return db
    except Exception as e:
        print(e)

In [4]:
# Connect to the db
db = get_database()

In [5]:
# Test Connection
serverStatusResult=db.command("serverStatus")
print(serverStatusResult["version"])

5.0.6


In [6]:
# Gets the coinpairs to process from the database
def get_coinpairs():
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = list(binance_coinpairs_df['pair'])
    return coinpair_list
coinpair_list = get_coinpairs()

In [7]:
# Function to get the timeframes stored in the database
def get_binance_timeframes_list(db):
    # Get the timeframes from the Database
    db_binance_timeframes = db["binance_timeframes"].find()

    # Convert the dictionary objects to dataframe
    binance_timeframes_df = pd.DataFrame(db_binance_timeframes)

    # Return the timeframes from the db
    return list(binance_timeframes_df['timeframe'])

binance_timeframes_list = get_binance_timeframes_list(db)
print(binance_timeframes_list)

['1m', '5m', '30m', '1h', '1d']


In [8]:
# Get latest inserted kline date for each Coin Pair
def get_pair_data(binance_timeframes_list, coinpair_list):
    complete_pair_tf = []
    from_timestamp = "3 months ago UTC"
    for timeframe in binance_timeframes_list:
        for pair in coinpair_list:
            # Get the coinpairs from the Database
            collection_name = pair+"_"+timeframe
            latest = db[collection_name].find().sort('open_time', -1 ).limit(1)
            # Exists, update collection by setting from_timestamp to lastest timestamp
            if collection_name in db.list_collection_names():
                complete_pair_tf.append([timeframe, pair, latest[0]["open_time"].strftime("%m/%d/%Y, %H:%M:%S"),collection_name, "is_update"])
            # Does not exists, import from csv
            else:
                complete_pair_tf.append([timeframe, pair, from_timestamp, collection_name, "is_new"])
    return complete_pair_tf

In [9]:
# Function to get the kline data from Binance
async def get_coinpair_kline(pair, timeframe, from_timestamp):
    client = await AsyncClient.create(BINANCE_API_KEY, BINANCE_API_SECRET)
    if timeframe == "1m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1MINUTE, from_timestamp)
    elif timeframe == "5m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_5MINUTE, from_timestamp)
    elif timeframe == "30m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_30MINUTE, from_timestamp)
    elif timeframe == "1h":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1HOUR, from_timestamp)
    elif timeframe == "1d":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1DAY, from_timestamp)
    else:
        return
    await client.close_connection()
    return klines

In [10]:
async def get_binance_data(complete_pair_tf):
    headers=["open_time", "open", "high", "low","close","volume","close_time","quote_asset_volume","numer_trades","taker_base_volume","taker_quote_volume","ignore"]
    for query_pair in complete_pair_tf:
        collection_name = query_pair[1]+"_"+query_pair[0]
        print(f"Processing {collection_name}..")
        kline_list = await get_coinpair_kline(query_pair[1], query_pair[0], query_pair[2])
        # If it is updating the db then delete the first item as it repeats
        if query_pair[4] == "is_update":
            kline_list.pop(0)
            print(f"Updating {len(kline_list)} items..")
        else:
            print(f"Adding {len(kline_list)} items..")
        if len(kline_list) > 0:
            kline_df = pd.DataFrame(kline_list, columns=headers)
            kline_df['open_time'] = kline_df['open_time'].values.astype(dtype='datetime64[ms]')
            kline_df['close_time'] = kline_df['close_time'].values.astype(dtype='datetime64[ms]')
            kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]] = kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]].astype(float)
            kline_df = kline_df.drop(columns=['ignore'])
            kline_dict = kline_df.to_dict("records")
            #print(kline_dict[0])
            db[query_pair[3]].insert_many(kline_dict)
        print(f"Done :)")


In [11]:
# Get pair info to update DB
complete_pair_tf = get_pair_data(binance_timeframes_list, coinpair_list)
# print(complete_pair_tf)

In [12]:
# Update DB with the latest binance Data
await get_binance_data(complete_pair_tf)

Processing ADABUSD_1m..
Updating 1801 items..
Done :)
Processing SOLBUSD_1m..
Updating 1801 items..
Done :)
Processing BTCBUSD_1m..
Updating 1800 items..
Done :)
Processing ETHBUSD_1m..
Updating 1800 items..
Done :)
Processing LUNABUSD_1m..
Updating 1800 items..
Done :)
Processing DOGEBUSD_1m..
Updating 1800 items..
Done :)
Processing MATICBUSD_1m..
Updating 1799 items..
Done :)
Processing MANABUSD_1m..
Updating 1799 items..
Done :)
Processing ADABUSD_5m..
Updating 360 items..
Done :)
Processing SOLBUSD_5m..
Updating 360 items..
Done :)
Processing BTCBUSD_5m..
Updating 360 items..
Done :)
Processing ETHBUSD_5m..
Updating 360 items..
Done :)
Processing LUNABUSD_5m..
Updating 360 items..
Done :)
Processing DOGEBUSD_5m..
Updating 360 items..
Done :)
Processing MATICBUSD_5m..
Updating 360 items..
Done :)
Processing MANABUSD_5m..
Updating 360 items..
Done :)
Processing ADABUSD_30m..
Updating 60 items..
Done :)
Processing SOLBUSD_30m..
Updating 60 items..
Done :)
Processing BTCBUSD_30m..
Upd

In [13]:
coinpair_list

['ADABUSD',
 'SOLBUSD',
 'BTCBUSD',
 'ETHBUSD',
 'LUNABUSD',
 'DOGEBUSD',
 'MATICBUSD',
 'MANABUSD']

In [14]:
# Gets the coinpairs to process from the database
def get_keywords(coinpair):
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"pair": coinpair, "exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = list(binance_coinpairs_df['keywords'][0])
    
    return coinpair_list

# Gets the main coin name to process from the database
def get_main_coin(coinpair):
    # Get the coin name from the Database
    db_coinpairs = db["coinpairs"].find({"pair": coinpair, "exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = binance_coinpairs_df['main'][0]
    
    return coinpair_list

# Gets the coinpairs to process from the database
def get_coinpairs():
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = list(binance_coinpairs_df['pair'])
    return coinpair_list

# Save trends to Database
def update_trends_db(coin_name, trends_df):
    # Resets the index
    trends_df = trends_df.reset_index()
    
    # Update the coinpairs in the Database
    db["trends_"+coin_name].insert_many(trends_df.to_dict("records"))
    db["trends_"+coin_name].create_index([ ("date", -1) ])
    
    # Returns the coin pair
    return coinpair

# TODO REWRITE TO GET DATa FOR UPDATE
# Function to initialize the databse with the coinpairs registered
def update_google_trends(db, coinpairs):
    for coinpair in coinpairs:
        coin_name = get_main_coin(coinpair)
        collection_name="trends_"+coin_name
        if collection_name not in db.list_collection_names():  
            print(f"Pulling trends for {coin_name}")
            keywords = get_keywords(coinpair)
            populate_google_trends(coinpair, keywords, coin_name)
        else:
            print(f"Collection {collection_name} found, skipping..")
            
    print(f"Done pulling trends.")

In [15]:
#def update_google_trends():
#    print("yeah")


In [16]:
#update_google_trends()